In [1]:
# Import libraries and read csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

df = pd.read_csv('TechChallenge_Data.xlsx - Sheet1.csv')

In [2]:
# Copying dataframe
df_cleaned = df.copy()

# Checking for missing values
print(df_cleaned.isnull().sum())

# Creating Date Column and droppping redudant cols
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Year'].astype(str) + '-' + df_cleaned['Month_num'].astype(str),
format = '%Y-%m')

df_cleaned = df_cleaned.drop(columns = ['Month', 'Year', 'Month_num'])


# Creating Standardized Route Column and dropping redundant cols
df_cleaned['Route'] = df_cleaned['AustralianPort'] + ' - ' + df_cleaned['ForeignPort']
df_cleaned = df_cleaned.drop(columns = ['AustralianPort', 'ForeignPort'])


df_cleaned.head()



Month                     0
AustralianPort            0
ForeignPort               0
Country                   0
Passengers_In             0
Freight_In_(tonnes)       0
Mail_In_(tonnes)          0
Passengers_Out            0
Freight_Out_(tonnes)      0
Mail_Out_(tonnes)         0
Passengers_Total          0
Freight_Total_(tonnes)    0
Mail_Total_(tonnes)       0
Year                      0
Month_num                 0
dtype: int64


,Country,Passengers_In,Freight_In_(tonnes),Mail_In_(tonnes),Passengers_Out,Freight_Out_(tonnes),Mail_Out_(tonnes),Passengers_Total,Freight_Total_(tonnes),Mail_Total_(tonnes),Date,Route
0,New Zealand,1513,42.167,0.311,985,18.704,0.924,2498,60.871,1.235,1985-01-01,Adelaide - Auckland
1,Bahrain,12,0.000,0.000,5,0.033,0.000,17,0.033,0.000,1985-01-01,Adelaide - Bahrain
2,India,7,0.000,0.000,5,0.000,0.000,12,0.000,0.000,1985-01-01,Adelaide - Bombay
3,Germany,115,0.009,0.000,171,0.000,0.248,286,0.009,0.248,1985-01-01,Adelaide - Frankfurt
4,UK,1567,2.800,0.000,1472,10.618,2.487,3039,13.418,2.487,1985-01-01,Adelaide - London


In [3]:
# How has Overall Passenger Traffic changed across regions over time (1985-1989)?


# Getting Monthly Passengers (Route and Country)
monthly_country_trends = (df_cleaned
    .groupby([df_cleaned['Date'].dt.to_period('M'), 'Country'])
    ['Passengers_Total'].sum()
    .reset_index()
    .rename(columns={'Date': 'Month', 'Passengers_Total': 'Monthly_Passengers'})
    )

print('\nMonthly Passenger Traffic Trends by Country:\n')
print(monthly_country_trends.head())


monthly_route_trends = (df_cleaned
    .groupby([df_cleaned['Date'].dt.to_period('M'), 'Route'])
    ['Passengers_Total'].sum()
    .reset_index()
    .rename(columns={'Date': 'Month', 'Passengers_Total': 'Monthly_Passengers'})
    )


print('\nMonthly Passenger Traffic Trends by Route:\n')
print(monthly_route_trends.head())





Monthly Passenger Traffic Trends by Country:

     Month  Country  Monthly_Passengers
0  1985-01  Bahrain                 623
1  1985-01   Brunei                 841
2  1985-01   Canada                3178
3  1985-01    China                4654
4  1985-01   Cyprus                  57

Monthly Passenger Traffic Trends by Route:

     Month                 Route  Monthly_Passengers
0  1985-01   Adelaide - Auckland                2498
1  1985-01    Adelaide - Bahrain                  17
2  1985-01     Adelaide - Bombay                  12
3  1985-01  Adelaide - Frankfurt                 286
4  1985-01     Adelaide - London                3039


In [4]:
# SARIMA Model (Seasonal Auto-Regressive Integrated Moving Average)

# Importing library for SARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

# Deining a target route
target_route = 'Sydney - Singapore'

# Converting Month column to datetime
if isinstance(monthly_route_trends['Month'].dtype, pd.PeriodDtype):
    monthly_route_trends['Month'] = monthly_route_trends['Month'].dt.to_timestamp()
 
    
# 1-D Series for the Route (Month and Passengers) 
ts = (monthly_route_trends[monthly_route_trends['Route'] == target_route]
      .copy()
      .sort_values('Month')
      .set_index('Month')['Monthly_Passengers']
      .asfreq('MS')) 


# Use auto_arima library to auto tune SARIMA parameters
auto_model = auto_arima(
    ts,
    seasonal = True,
    m = 12,             
    stepwise = True,
    suppress_warnings = True,
    error_action = 'ignore',
    trace = False)

# Extract best parameters from auto model
p, d, q = auto_model.order
P, D, Q, s = auto_model.seasonal_order
print(f'Best orders found: order = {(p, d, q)}, seasonal_order = {(P, D, Q, s)}')


# Create and Fit SARIMA model with auto parameters
model = SARIMAX(ts,
    order = (p, d, q),
    seasonal_order = (P, D, Q, s),
    enforce_stationarity = False,
    enforce_invertibility = False)

model_fit = model.fit(disp = False)


# Forecasting next years data for specificed route
steps = 12
forecast = model_fit.get_forecast(steps = steps).predicted_mean
print(forecast.round(0))




Best orders found: order = (0, 1, 1), seasonal_order = (1, 0, 0, 12)
1989-07-01    31386.0
1989-08-01    29913.0
1989-09-01    30572.0
1989-10-01    31438.0
1989-11-01    31249.0
1989-12-01    32193.0
1990-01-01    33457.0
1990-02-01    30836.0
1990-03-01    33354.0
1990-04-01    31364.0
1990-05-01    28930.0
1990-06-01    30336.0
Freq: MS, Name: predicted_mean, dtype: float64


In [5]:
# Evaluating Model Performance


# Train(everything until last year data) test (last year data) split
train = ts.iloc[:-12]
test = ts.iloc[-12:]

# Creating SAMIRA Model using training data
model_eval = SARIMAX(train, order = (p, d, q), seasonal_order = (P, D, Q, s),
                    enforce_stationarity = False,
                    enforce_invertibility = False)

# Fitting model on training data
fit_eval = model_eval.fit(disp = False)

pred = fit_eval.forecast(steps = len(test))


mae  = float(np.mean(np.abs(test.values - pred.values)))
mape = float(np.mean(np.abs((test.values - pred.values) / np.clip(test.values, 1e-9, None))) * 100)
smape = float(np.mean(2.0 * np.abs(test.values - pred.values) /
                      np.clip(np.abs(test.values) + np.abs(pred.values), 1e-9, None)) * 100)

print(f'MAE  = {round(mae, 0)}')
print(f'MAPE = {round(mape, 2)}%')
print(f'sMAPE = {round(smape, 2)}%')

MAE  = 862.0
MAPE = 2.85%
sMAPE = 2.83%
